<a href="https://colab.research.google.com/github/RoshanKohli1402/Lab_works/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.6 MB/s eta 0:00:00


In [2]:
import pdfplumber
import json
import os
pdf_path = "/content/form10-k.pdf"
output_path = "/content/extracted_pages.json"

print("PDF exists:", os.path.exists(pdf_path))

pages_data = []

with pdfplumber.open(pdf_path) as pdf:
    print("Total pages:", len(pdf.pages))
    for page_number, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if text is None:
            text = ""
        pages_data.append({
            "page": page_number,
            "text": text.strip()
        })

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(pages_data, f, indent=2, ensure_ascii=False)

print("Extraction complete")
print("Saved to:", output_path)


PDF exists: True
Total pages: 19
Extraction complete
Saved to: /content/extracted_pages.json


In [3]:
import json
import re

input_path = "/content/extracted_pages.json"
output_path = "/content/extracted_clauses.json"

# Keywords that usually indicate obligations
OBLIGATION_KEYWORDS = [
    "must",
    "shall",
    "is required to",
    "are required to",
    "will be required to",
    "may not",
    "shall not"
]

def contains_obligation(sentence):
    sentence_lower = sentence.lower()
    return any(keyword in sentence_lower for keyword in OBLIGATION_KEYWORDS)

# Load extracted pages
with open(input_path, "r", encoding="utf-8") as f:
    pages = json.load(f)

clauses = []

for page in pages:
    page_number = page["page"]
    text = page["text"]

    # Split text into sentences (simple split)
    sentences = re.split(r'(?<=[.!?])\s+', text)

    for sentence in sentences:
        if contains_obligation(sentence):
            clauses.append({
                "page": page_number,
                "clause": sentence.strip()
            })

# Save extracted clauses
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(clauses, f, indent=2, ensure_ascii=False)

print("Total clauses extracted:", len(clauses))
print("Saved to:", output_path)


Total clauses extracted: 45
Saved to: /content/extracted_clauses.json


In [4]:
import json

input_path = "/content/extracted_clauses.json"
output_path = "/content/mapped_clauses.json"

BUSINESS_UNIT_RULES = {
    "Finance / Accounting": [
        "financial reporting",
        "accounting",
        "disclosure",
        "financial statement"
    ],
    "Risk Management": [
        "risk",
        "liquidity",
        "capital",
        "exposure"
    ],
    "Compliance / Internal Audit": [
        "compliance",
        "internal control",
        "audit",
        "regulatory"
    ],
    "Technology / IT": [
        "system",
        "technology",
        "cyber",
        "data"
    ]
}

def map_business_unit(clause_text):
    clause_lower = clause_text.lower()
    for unit, keywords in BUSINESS_UNIT_RULES.items():
        for keyword in keywords:
            if keyword in clause_lower:
                return unit
    return "Unclassified"

# Load clauses
with open(input_path, "r", encoding="utf-8") as f:
    clauses = json.load(f)

mapped = []

for item in clauses:
    mapped.append({
        "page": item["page"],
        "clause": item["clause"],
        "business_unit": map_business_unit(item["clause"])
    })

# Save result
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(mapped, f, indent=2, ensure_ascii=False)

print("Mapped clauses:", len(mapped))
print("Saved to:", output_path)


Mapped clauses: 45
Saved to: /content/mapped_clauses.json


In [5]:
import json

input_path = "/content/mapped_clauses.json"
output_path = "/content/scored_clauses.json"

RISK_SCORES = {
    "Risk Management": 3,
    "Finance / Accounting": 2,
    "Compliance / Internal Audit": 2,
    "Technology / IT": 1,
    "Unclassified": 1
}

with open(input_path, "r", encoding="utf-8") as f:
    clauses = json.load(f)

scored = []

for item in clauses:
    unit = item["business_unit"]
    risk_score = RISK_SCORES.get(unit, 1)

    scored.append({
        "page": item["page"],
        "clause": item["clause"],
        "business_unit": unit,
        "risk_score": risk_score
    })

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(scored, f, indent=2, ensure_ascii=False)

print("Scored clauses:", len(scored))
print("Saved to:", output_path)


Scored clauses: 45
Saved to: /content/scored_clauses.json


In [6]:
import json

input_path = "/content/scored_clauses.json"
output_path = "/content/final_actions.json"

HIGH_RISK_THRESHOLD = 3

with open(input_path, "r", encoding="utf-8") as f:
    clauses = json.load(f)

# Agent logic: sort + prioritize
clauses_sorted = sorted(
    clauses,
    key=lambda x: x["risk_score"],
    reverse=True
)

final_actions = []

for item in clauses_sorted:
    priority = "HIGH" if item["risk_score"] >= HIGH_RISK_THRESHOLD else "MEDIUM"

    final_actions.append({
        "priority": priority,
        "risk_score": item["risk_score"],
        "business_unit": item["business_unit"],
        "page": item["page"],
        "clause": item["clause"]
    })

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(final_actions, f, indent=2, ensure_ascii=False)

print("Final actions generated:", len(final_actions))
print("Saved to:", output_path)


Final actions generated: 45
Saved to: /content/final_actions.json


In [7]:
import json
import pandas as pd
import re

# Load extracted pages
with open("/content/extracted_pages.json", "r", encoding="utf-8") as f:
    pages = json.load(f)

rows = []

for page in pages:
    page_num = page["page"]
    text = page["text"]

    sentences = re.split(r'(?<=[.!?])\s+', text)
    for s in sentences:
        s = s.strip()
        if len(s) > 30:
            rows.append({
                "page": page_num,
                "sentence": s
            })

df = pd.DataFrame(rows)
print("Total sentences:", len(df))
df.head()


Total sentences: 182


,page,sentence
0,1,OMB APPROVAL\nUNITED STATES\nOMB Number: 3235-...
1,1,20549 Estimated average burden\nhours per resp...
2,1,(1) This Form shall be used for annual reports...
3,1,78m or 78o(d)) (the “Act”) for which no\nother...
4,1,This Form also shall be used for transition re...


In [8]:
OBLIGATION_KEYWORDS = [
    "must", "shall", "is required to", "are required to",
    "will be required to", "may not", "shall not"
]

def rule_label(sentence):
    s = sentence.lower()
    return 1 if any(k in s for k in OBLIGATION_KEYWORDS) else 0

df["label"] = df["sentence"].apply(rule_label)
df.head()


,page,sentence,label
0,1,OMB APPROVAL\nUNITED STATES\nOMB Number: 3235-...,0
1,1,20549 Estimated average burden\nhours per resp...,0
2,1,(1) This Form shall be used for annual reports...,1
3,1,78m or 78o(d)) (the “Act”) for which no\nother...,0
4,1,This Form also shall be used for transition re...,1


In [9]:
df_sample = df.sample(150, random_state=42).copy()
df_sample


,page,sentence,label
19,2,"(3) Attention is directed to Rule 12b-20, whic...",1
42,3,If any information required by Part I or Part ...,1
154,15,Principal Accountant Fees and Services.,0
98,10,The listed documents should be\nclearly descri...,0
146,15,PART III\n[See General Instruction G(3)]\nItem...,0
...,...,...,...
134,13,(b) Furnish the information required by Item 4...,0
167,16,(a) List the following documents filed as a pa...,0
166,16,Exhibits and Financial Statement Schedules.,0
63,5,(b) Such registrants may omit the list of subs...,0


In [10]:
df_sample.to_csv("/content/training_data.csv", index=False)
print("Saved training dataset")

Saved training dataset


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

data = pd.read_csv("/content/training_data.csv")

X_train, X_test, y_train, y_test = train_test_split(
    data["sentence"],
    data["label"],
    test_size=0.25,
    random_state=42,
    stratify=data["label"]
)

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.9
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      1.00      0.85        28
           1       0.00      0.00      0.00        10

    accuracy                           0.74        38
   macro avg       0.37      0.50      0.42        38
weighted avg       0.54      0.74      0.63        38



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
def hybrid_classify(sentence):
    s = sentence.lower()

    # Rule-based high confidence
    if any(k in s for k in OBLIGATION_KEYWORDS):
        return 1, "rule", 1.0

    # ML fallback
    vec = vectorizer.transform([sentence])
    prob = model.predict_proba(vec)[0][1]

    if prob >= 0.8:
        return 1, "ml", prob
    else:
        return 0, "ml", prob


In [13]:
hybrid_results = []

for _, row in df.iterrows():
    label, source, confidence = hybrid_classify(row["sentence"])
    if label == 1:
        hybrid_results.append({
            "page": row["page"],
            "clause": row["sentence"],
            "decision_source": source,
            "confidence": round(confidence, 3)
        })

len(hybrid_results)


45

In [14]:
with open("/content/ml_extracted_clauses.json", "w", encoding="utf-8") as f:
    json.dump(hybrid_results, f, indent=2, ensure_ascii=False)

print("ML-enhanced clauses saved")

ML-enhanced clauses saved


In [15]:
import json

config = {
    "jurisdiction": "US",
    "regulation": "SEC_10K",
    "language": "en",

    "obligation_keywords": [
        "must",
        "shall",
        "is required to",
        "are required to",
        "will be required to",
        "shall not",
        "may not"
    ],

    "business_unit_rules": {
        "Finance / Accounting": [
            "financial reporting",
            "accounting",
            "disclosure",
            "financial statement"
        ],
        "Risk Management": [
            "risk",
            "liquidity",
            "capital",
            "exposure"
        ],
        "Compliance / Internal Audit": [
            "compliance",
            "internal control",
            "audit",
            "regulatory"
        ],
        "Technology / IT": [
            "system",
            "technology",
            "cyber",
            "data"
        ]
    },

    "risk_scores": {
        "Risk Management": 3,
        "Finance / Accounting": 2,
        "Compliance / Internal Audit": 2,
        "Technology / IT": 1,
        "Unclassified": 1
    },

    "ml_threshold": 0.8
}

with open("/content/regulation_config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

print("Global regulation configuration saved.")


Global regulation configuration saved.


In [16]:
with open("/content/regulation_config.json", "r", encoding="utf-8") as f:
    CONFIG = json.load(f)

OBLIGATION_KEYWORDS = CONFIG["obligation_keywords"]
BUSINESS_UNIT_RULES = CONFIG["business_unit_rules"]
RISK_SCORES = CONFIG["risk_scores"]
ML_THRESHOLD = CONFIG["ml_threshold"]

print("Loaded config for:", CONFIG["jurisdiction"], CONFIG["regulation"])


Loaded config for: US SEC_10K


In [17]:
def hybrid_classify(sentence):
    s = sentence.lower()

    # Rule-based high confidence
    if any(k in s for k in OBLIGATION_KEYWORDS):
        return 1, "rule", 1.0

    # ML fallback
    vec = vectorizer.transform([sentence])
    prob = model.predict_proba(vec)[0][1]

    if prob >= ML_THRESHOLD:
        return 1, "ml", prob
    else:
        return 0, "ml", prob


In [18]:
def map_business_unit(clause_text):
    clause_lower = clause_text.lower()
    for unit, keywords in BUSINESS_UNIT_RULES.items():
        for keyword in keywords:
            if keyword in clause_lower:
                return unit
    return "Unclassified"


In [19]:
def get_risk_score(business_unit):
    return RISK_SCORES.get(business_unit, 1)
